In [1]:
from pyquery import PyQuery as pq
import requests
import re
import csv
from string import capwords
# import uucsv

In [2]:
votelist = []    # to be billno, chamber, membername, vote, points
floorvotevalue = 3

In [3]:
billlist = []
with open('billlist.csv', newline='') as csvfile:
     reader = csv.DictReader(csvfile)
     for row in reader:
        if row['position'] == "bad":
            row['multiplier'] = -1
        elif row['position'] == "good":
            row['multiplier'] = 1
        else:
            print("Improper 'position' found for " + row['billno'] + ". What did you do?")
        billlist.append(row)

In [ ]:
# html = requests.get("https://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17855&BillId=60058&SessionId=86&")
# yas = 



In [15]:
for row in billlist:
    for lowchamber in ("house", "senate"):
        if len(row[lowchamber]) > 0:   # If we have a valid URL
            billno = row['billno']
            chamber = capwords(lowchamber)
            print("Working on " + billno + " for the " + chamber)
            multiplier = row['multiplier']
            sourceurl = row[lowchamber]
            r = requests.get(sourceurl)
            html = r.content 
            yeas = pq(html)("span#ctl00_MainContent_lblYeas").text().strip()
            nays = pq(html)("span#ctl00_MainContent_lblNays").text().strip()
            abstained = pq(html)("span#ctl00_MainContent_lblAbstained").text().strip()
            missed = pq(html)("span#ctl00_MainContent_lblMissed").text().strip()
            total = pq(html)("span#ctl00_MainContent_lblTotal").text().strip()
            ryeas = 0   # Start capturing real votes cast to compare.
            rnays = 0
            rsomething = 0
            rtotal = 0
            mainvotes = pq(html)("div.member-votes")
            for pol in pq(mainvotes)("li"):
                vote = pq(pq(pol)("span")[0]).text().strip()
                member = pq(pq(pol)("span")[1]).text().strip()
                # print(f"{vote}: {member}")
                points = 0
                if vote == "Y":
                    points = floorvotevalue * multiplier
                    ryeas += 1
                    rtotal += 1
                elif vote == "N":
                    points = floorvotevalue * multiplier * -1
                    rnays += 1
                    rtotal += 1
                else:
                    print(f"Vote {vote} found for {member} on {billno} in {chamber}")
                    # rtotal += 1   # Yes, No only counted toward total apparently
                    rsomething += 1
                entry = [billno, chamber, member, vote, str(points)]
                # print(member, vote, memberid)
                votelist.append(entry)
            if "Vote After Roll Call" in str(pq(html)):
                print("After-roll call votes need to be manually added for:\n\t"+ sourceurl)
            print(f"{yeas}-{nays}-{abstained}-{missed}-{total} votes from summary found. Y-N-A-M-Total")
            print(f"{ryeas}-{rnays}-{rsomething}-{rtotal} individual votes found Y-N-Something-Total")
    
    

Working on HB273 for the House
Vote - found for Altman on HB273 in House
Vote - found for Avila on HB273 in House
Vote - found for Boyd on HB273 in House
Vote - found for Davis on HB273 in House
Vote - found for Good on HB273 in House
Vote - found for Hager on HB273 in House
Vote - found for Plakon on HB273 in House
Vote - found for Rodrigues, R. on HB273 in House
Vote - found for Toledo on HB273 in House
Vote - found for Vacant 33 on HB273 in House
After-roll call votes need to be manually added for:
	https://www.myfloridahouse.gov/Sections/Bills/floorvote.aspx?VoteId=17855&BillId=60058&SessionId=86&
108-0-0-10-108 votes from summary found. Y-N-A-M-Total
108-0-10-108 individual votes found Y-N-something-Total
Working on HB459 for the House
Vote - found for Altman on HB459 in House
Vote - found for Hager on HB459 in House
Vote - found for Metz on HB459 in House
Vote - found for Vacant 33 on HB459 in House
After-roll call votes need to be manually added for:
	https://www.myfloridahouse.

In [11]:
# OK, so there can be fewer than 120 reps -- someone died, quit? Not marked absent. GOod to know.
# If number of voters is less than or equal to 40, it's Senate; otherwise, House

In [12]:
with open('autovotes.csv', 'w', newline='') as csvfile:
    put = csv.writer(csvfile)
    header = ["billno", "chamber", "member", "vote", "points"]
    put.writerow(header)
    for entry in votelist:
        put.writerow(entry)